In [1]:
pip install pandas dotenv requests streamlit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
from typing import Optional

load_dotenv()
api_key =os.getenv('openFDA_API_KEY')
api_key is None

True

In [8]:
# Count adverse events by drug name within the date range
url = "https://api.fda.gov/drug/event.json?search=receivedate:[20200101+TO+20241231]&count=patient.drug.medicinalproduct.exact"
data = fetch_api_data(url, "No Drug")

df = pd.DataFrame(data["results"], columns=["term", "count"])
df.columns = ["Drug Name", "Adverse Event Count"]

# Clean data
df["Drug Name"] = df["Drug Name"].str.replace(r"\.$", "", regex=True)  # Remove trailing periods
df["Drug Name"] = df["Drug Name"].str.strip().str.upper()  # Standardize format
df = df.dropna(subset=["Drug Name", "Adverse Event Count"])  # Remove nulls
df["Adverse Event Count"] = pd.to_numeric(df["Adverse Event Count"], errors="coerce").fillna(0).astype(int)  # Ensure counts are integers
df = df.drop_duplicates(subset=["Drug Name"])  # Remove duplicates
df

,Drug Name,Adverse Event Count
0,ZANTAC,316588
1,DUPIXENT,266046
2,HUMIRA,213721
3,REVLIMID,152042
4,PREDNISONE,142546
...,...,...
95,JARDIANCE,32352
96,LYRICA,31794
97,ACYCLOVIR,31565
98,RANITIDINE CAPSULE,31504


In [9]:
# Count adverse events by patient age group (within date range)
# Fetch patient age data
url = "https://api.fda.gov/drug/event.json?search=receivedate:[20200101+TO+20241231]&count=patient.patientonsetage"
data = fetch_api_data(url, "No Age Data")

# Build DataFrame
df_age = pd.DataFrame(data["results"], columns=["term", "count"])
df_age.columns = ["Patient Age", "Adverse Event Count"]

# Clean data
df_age["Patient Age"] = pd.to_numeric(df_age["Patient Age"], errors="coerce")  # Ensure numeric age
df_age = df_age.dropna(subset=["Patient Age", "Adverse Event Count"])  # Drop rows with nulls
df_age["Adverse Event Count"] = pd.to_numeric(df_age["Adverse Event Count"], errors="coerce").fillna(0).astype(int)
df_age = df_age[df_age["Patient Age"] > 0]  # Filter out invalid ages
df_age = df_age.drop_duplicates(subset=["Patient Age"])  # Remove duplicates

df


,Drug Name,Adverse Event Count
0,ZANTAC,316588
1,DUPIXENT,266046
2,HUMIRA,213721
3,REVLIMID,152042
4,PREDNISONE,142546
...,...,...
95,JARDIANCE,32352
96,LYRICA,31794
97,ACYCLOVIR,31565
98,RANITIDINE CAPSULE,31504


In [6]:
def fetch_api_data(url, drug_name: Optional[str] = None):
    try:
        payload = {}

        headers = {'api_key': api_key}
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data for {drug_name}: {e}")
        return {"results": []}


In [ ]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter


st.write("Hello World")
x = st.text_input("Favourite movie?")

2025-05-06 13:35:48.138 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.140 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35:48.148 Session state does not function when running a script without `streamlit run`
2025-05-06 13:35:48.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:35